In [26]:
%pwd

'd:\\chatbot\\End-to-End-Medical-Chatbot-Generative-AI\\research'

In [56]:
import os
os.chdir("../")

In [57]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [58]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [68]:
from langchain_community.document_loaders import PyPDFLoader

def load_single_pdf(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    return documents

extracted_data = load_single_pdf(r'D:\chatbot\End-to-End-Medical-Chatbot-Generative-AI\Data\clinicians-guide-to-cognitive-rehabilitation-in-mild-traumatic-brain-injury.pdf')


In [69]:
# extracted_data

In [70]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [71]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 561


In [72]:
from langchain.embeddings import HuggingFaceEmbeddings

In [73]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [74]:
embeddings = download_hugging_face_embeddings()

In [ ]:
pip install --upgrade langchain langchain-community sentence-transformers huggingface-hub


In [75]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [37]:
# query_result

In [86]:
from dotenv import load_dotenv
load_dotenv()

False

In [78]:
import os
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [79]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-1kb2huf.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [80]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [81]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [82]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [84]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [22]:
pip install --upgrade langchain langchain-community sentence-transformers huggingface-hub

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 24.1 MB/s eta 0:00:00

  Attempting uninstall: huggingface-hub

    Found existing installation: huggingface-hub 0.32.4

    Uninstalling huggingface-hub-0.32.4:

      Successfully uninstalled huggingface-hub-0.32.4

   ---------------------------------------- 0/3 [huggingface-hub]
   ---------------------------------------- 0/3 [huggingface-hub]
  Attempting uninstall: langchain-core
   ---------------------------------------- 0/3 [huggingface-hub]
    Found existing installation: langchain-core 0.3.64
   ---------------------------------------- 0/3 [huggingface-hub]
   ------------- -------------------------- 1/3 [langchain-core]
    Uninstalling langchain-core-0.3.64:
   ------------- -------------------------- 1/3 [langchain-core]
      Successfully uninstalled langchain-core-0.3.64
   ------------- -------------------------- 1/3 [langchain-core]
   ---------

In [ ]:
import os
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENROUTER_API_KEY=os.environ.get('OPENROUTER_API_KEY')

In [ ]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENROUTER_API_KEY"] = OPENROUTER_API_KEY


In [ ]:
import os
from dotenv import load_dotenv  # Install with: pip install python-dotenv

# Load keys from .env file
load_dotenv()  

# Access keys directly
pinecone_key = os.getenv("PINECONE_API_KEY")
openrouter_key = os.getenv("OPENROUTER_API_KEY")

# Verify keys exist
if not openai_key:
    raise ValueError("OPENROUTER_API_KEY not found in environment")

In [ ]:
import os

# Get keys from pre-configured system environment variables
os.environ["PINECONE_API_KEY"] = os.environ.get("PINECONE_API_KEY", "")
os.environ["OPENROUTER_API_KEY"] = os.environ.get("OPENROUTER_API_KEY", "")

In [92]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [93]:
docsearch

In [94]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [95]:
retrieved_docs = retriever.invoke("What is Acne?")

In [96]:
retrieved_docs

[Document(id='c0705383-55a6-4500-a87d-2b7bede5b444', metadata={'creationdate': '2016-12-30T07:22:15+00:00', 'creator': 'Word', 'keywords': '', 'moddate': '2017-07-03T08:28:11-04:00', 'page': 12.0, 'page_label': '13', 'producer': 'Mac OS X 10.11.6 Quartz PDFContext', 'source': 'D:\\chatbot\\End-to-End-Medical-Chatbot-Generative-AI\\Data\\clinicians-guide-to-cognitive-rehabilitation-in-mild-traumatic-brain-injury.pdf', 'title': 'Clinician’s Guide to Cognitive Rehabilitation in Mild Traumatic Brain Injury: Application for Military Service Members and Veterans', 'total_pages': 107.0, 'uploaddate': '7/3/2017'}, page_content='A\tlarge\tbody\tof\tliterature\taCributes\tpersistent\tcogni.ve\tsymptoms\taTer\tmTBI\tto\tproblems\t\t\nsuch\tas\theadaches,\tpain,\tsleep\tdisturbance,\tdepression,\tpost-deployment\tstress,\tor\tstress\t\ndisorders.\tIt\tis\toTen\tnot\tpossible\tto\taccurately\tiden.fy\thow\tmuch\teach\tcondi.on\tcontributes\t\nto\tongoing\tcogni.ve\tdiﬃcul.es.\tRegardless\tof\tthe\t

In [97]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [99]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [100]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [41]:
!pip install PyMuPDF


   ---------------------------------------- 0.0/18.5 MB ? eta -:--:--
   ------ --------------------------------- 2.9/18.5 MB 24.0 MB/s eta 0:00:01
   ----------- ---------------------------- 5.2/18.5 MB 13.3 MB/s eta 0:00:01
   ---------------- ----------------------- 7.6/18.5 MB 13.1 MB/s eta 0:00:01
   -------------------- ------------------- 9.7/18.5 MB 11.6 MB/s eta 0:00:01
   ------------------------ --------------- 11.5/18.5 MB 11.8 MB/s eta 0:00:01
   -------------------------------- ------- 15.2/18.5 MB 12.3 MB/s eta 0:00:01
   -------------------------------------- - 17.8/18.5 MB 12.4 MB/s eta 0:00:01
   ---------------------------------------- 18.5/18.5 MB 12.1 MB/s eta 0:00:00


In [101]:
import fitz  # PyMuPDF

def extract_pdf_text(pdf_path):
    doc = fitz.open(pdf_path)
    return " ".join([page.get_text() for page in doc])


In [102]:
def chunk_text(text, max_length=1000):
    return [text[i:i+max_length] for i in range(0, len(text), max_length)]

In [105]:
pdf_text = extract_pdf_text("D:\chatbot\End-to-End-Medical-Chatbot-Generative-AI\Data\clinicians-guide-to-cognitive-rehabilitation-in-mild-traumatic-brain-injury.pdf")
chunks = chunk_text(pdf_text)

# Use only the first chunk or combine multiple if needed
context = chunks[0]

data = {
    "model": "mistralai/mistral-7b-instruct:free",
    "messages": [
        {"role": "system", "content": f"You are a helpful assistant. Only use the following context to answer:\n\n{context}"},
        {"role": "user", "content": "What is Transition to Self-Management"}
    ]
}


In [106]:
import requests

api_key = "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"  # Your OpenRouter API key

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json",
    "HTTP-Referer": "https://chat.openai.com",
    "X-Title": "MyApp"
}

response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=data)

if response.status_code == 200:
    print(response.json()["choices"][0]["message"]["content"])
else:
    print("Error:", response.status_code, response.text)


 The "Transition to Self-Management" is not explicitly defined in the provided context. However, given the context of the Clinician's Guide to Cognitive Rehabilitation in Mild Traumatic Brain Injury, it is likely that Transition to Self-Management refers to the process where the individuals with mild traumatic brain injury (mTBI) are helped to take on greater responsibility for managing their cognitive impairments and recovery, with reduced supervision from healthcare professionals. This process involves teaching them strategies and skills that enable them to effectively self-manage their condition, adapt to challenges, and maximize their independence.
